In [3]:
import pennylane as qml
import numpy as np

def gradient_200(weights, dev):
    r"""This function must compute the gradient *and* the Hessian of the variational
    circuit using the parameter-shift rule, using exactly 51 device executions.
    The code you write for this challenge should be completely contained within
    this function between the # QHACK # comment markers.
    Args:
        weights (array): An array of floating-point numbers with size (5,).
        dev (Device): a PennyLane device for quantum circuit execution.
    Returns:
        tuple[array, array]: This function returns a tuple (gradient, hessian).
            * gradient is a real NumPy array of size (5,).
            * hessian is a real NumPy array of size (5, 5).
    """

    @qml.qnode(dev, interface=None)
    def circuit(w):
        for i in range(3):
            qml.RX(w[i], wires=i)

        qml.CNOT(wires=[0, 1])
        qml.CNOT(wires=[1, 2])
        qml.CNOT(wires=[2, 0])

        qml.RY(w[3], wires=1)

        qml.CNOT(wires=[0, 1])
        qml.CNOT(wires=[1, 2])
        qml.CNOT(wires=[2, 0])

        qml.RX(w[4], wires=2)

        return qml.expval(qml.PauliZ(0) @ qml.PauliZ(2))

    gradient = np.zeros([5], dtype=np.float64)
    grad=np.zeros([5], dtype=np.float64)
    hessian = np.zeros([5, 5], dtype=np.float64)

    # QHACK #
    def parameter_shift_term(qnode, weights, i):
        shifted = weights.copy()
        shifted[i] += np.pi/4
        forward = qnode(shifted)  # forward evaluation

        shifted[i] -= np.pi/2
        backward = qnode(shifted) # backward evaluation
        return 0.5 * ((forward - backward)/(np.sin(np.pi/4)))
    
    def parameter_shift(qnode, params):

        for i in range(len(params)):
            gradient[i] = parameter_shift_term(qnode, weights, i)
            grad[i]=parameter_shift_term(qnode,gradient,i)
        for i in range(len(gradient)):
            for j in range(len(gradient)):
                if(i==j):
                    hessian[i][j]=grad[i]
                else:
                    hessian[i][j]=parameter_shift_term(qnode,gradient,i)
        
        return gradient,hessian
    
    gradient,hessian=parameter_shift(circuit,weights)
    
        # iterate over dimensions
        # apply gradient again to every component of the first derivative.
        

                
    # QHACK #

    return gradient, hessian, circuit.diff_options["method"]



In [4]:
weights = np.array([0.1,0.2,0.1,0.2,0.7], float)

dev = qml.device("default.qubit", wires=3)
gradient, hessian, diff_method = gradient_200(weights, dev)

print(
    *np.round(gradient, 10),
        sep=","
    )


0.0012756024,-0.7668909241,-0.1890228368,-0.0374176229,-0.7914937431


In [5]:
print(*np.round(hessian,10),sep=",")


[-0.0000e+00 -6.3787e-06 -6.3787e-06 -6.3787e-06 -6.3787e-06],[0.98176049 0.69389984 0.98176049 0.98176049 0.98176049],[-0.02374178 -0.02374178  0.13530088 -0.02374178 -0.02374178],[-0.15171627 -0.15171627 -0.15171627  0.         -0.15171627],[0.97683747 0.97683747 0.97683747 0.97683747 0.97683747]


a a
a b
a c
a d
a e
b a
b b
b c
b d
b e
c a
c b
c c
c d
c e
d a
d b
d c
d d
d e
e a
e b
e c
e d
e e
